In [44]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import re
import csv
import os

chrome_options = Options()
#chrome_options.add_argument("--headless")

driver = webdriver.Chrome(chrome_options=chrome_options)

In [45]:
url = 'https://www.youtube.com/user/MRCURTIS137599/playlists'

driver.get(url)

In [46]:
names = [n.get_attribute('title') for n in driver.find_elements_by_xpath('//*[@id="video-title"]')]
playlist_urls = [p.get_attribute('href') for p in driver.find_elements_by_xpath('//*[@id="view-more"]/a')]
playlist = dict(zip([i for i in range(len(names))], zip(names, playlist_urls)))

In [47]:
playlist

{0: ('Micro',
  'https://www.youtube.com/playlist?list=PLvEaFQJe3P3IJzXrBqFSrqAUPGwkUMw9a'),
 1: ('Funcking',
  'https://www.youtube.com/playlist?list=PLvEaFQJe3P3LJh1VoFnzX69Y0YpFzXqBQ'),
 2: ('dl',
  'https://www.youtube.com/playlist?list=PLvEaFQJe3P3JmSZMF4GH8gxS6Usuz353r'),
 3: ('Favorites',
  'https://www.youtube.com/playlist?list=FL4Y7L4F7miXzhdLdlJuN2fQ'),
 4: ('touk',
  'https://www.youtube.com/playlist?list=PLvEaFQJe3P3LB0JvI4u_AIMC4hk7bU8--')}

In [48]:
print('Hello ZoziPop!\nHere, your playlists.\nWhich one do you want to update ?\n')
for n, url in playlist.items():
    print(f'[{n}] -> {url[0]}')

choice = int(input('Enter the number corresponding to the playlist:\n'))

Hello ZoziPop!
Here, your playlists.
Which one do you want to update ?

[0] -> Micro
[1] -> Funcking
[2] -> dl
[3] -> Favorites
[4] -> touk


In [49]:
url = playlist[choice][1]
driver.get(url)

In [50]:
musics = []
nb_music = driver.find_element_by_xpath('//*[@id="stats"]/yt-formatted-string[1]/span[1]').text
while len(musics) != int(nb_music):
    driver.execute_script("window.scrollTo(0, 100000);")
    musics = driver.find_elements_by_xpath('//*[@id="content"]/a')

comp = re.compile('^(.+)&list')
musics_url = [comp.search(music.get_attribute('href')).group(1) for music in musics] 

In [51]:
with open(f'{playlist[choice][0]}/already_download.txt', 'r') as f:
    already_dl = [line.rstrip() for line in f]

with open(f'{playlist[choice][0]}/already_download.txt', 'w') as f:
    for url in musics_url:
        f.write(f"{url}\n")

musics_url = list(set(musics_url) - set(already_dl))

with open(f'{playlist[choice][0]}/urls_to_download.txt', 'w') as f:
    for url in musics_url:
        f.write(f"{url}\n")

In [52]:
os.system(f'python3 /usr/local/bin/youtube-dl -a ./{playlist[choice][0]}/urls_to_download.txt -o ~/Music/{playlist[choice][0]}/%(uploader)s/%(title)s.%(ext)s')

512